In [10]:
import optuna
from skimage import filters
from scipy.ndimage import gaussian_filter
from sklearn.metrics import f1_score
import colorcorrect.algorithm as cca
import mahotas as mh
import cellseg.src.image_worker as iw
import cellseg.src.convert_worker as cw
import cellseg.src.shape_descriptors as sd
import numpy as np

img = mh.imread("Images/BAL_images/2023_12_14_image_006.png")

file_path = 'Images/labels/cytoplasm/2023_12_14_image_006.txt'

with open(file_path, 'r') as f:
    data = [list(map(int, line.split())) for line in f]

cytoplasm_labels = np.array(data)

In [13]:
import pathlib

print(pathlib.Path("Images/BAL_images/2023_12_14_image_006.png").stem)

2023_12_14_image_006


In [6]:
def objective(trial):
    # Hyperparameters to optimize
    slope = trial.suggest_int('slope', 5, 20)
    limit = trial.suggest_int('limit', 500, 2000)
    samples = trial.suggest_int('samples', 100, 1000)
    sigma = trial.suggest_float('sigma', 1.0, 15.0)
    subwidth = trial.suggest_int('subwidth', 10, 30)
    subheight = trial.suggest_int('subweight', 10, 30)
    mask_size = trial.suggest_int('mask_size', 2, 5)
    iterations = trial.suggest_int('iterations', 1, 10)
    min_size = trial.suggest_int('min_size', 50, 300)
    
    # Step 1: Automatic color equalization
    img_c_corrected = cca.automatic_color_equalization(img, slope=slope, limit=limit, samples=samples)
    
    # Step 2: Gaussian Blur
    img_blurred = gaussian_filter(img_c_corrected, sigma=sigma)
    
    # Step 3: Luminance weighted gray world
    RGB_balanced = cca.luminance_weighted_gray_world(img_blurred, subwidth=subwidth, subheight=subheight)
    r2, g2, b2 = RGB_balanced[:, :, 0], RGB_balanced[:, :, 1], RGB_balanced[:, :, 2]
    
    # Step 4: Thresholding and segmentation
    thresholds = filters.threshold_multiotsu(g2)
    regions = np.digitize(g2, bins=thresholds)
    bin_cyto_nuclei = cw.convert_labeled_to_bin(regions, background=2)
    bin_cyto_nuclei = iw.close_holes_remove_noise(bin_cyto_nuclei, mask_size=mask_size, iterations=iterations)
    
    # Step 5: Label cytoplasm regions
    img_labeled_cytoplasm, nr_cytoplasm = mh.label(bin_cyto_nuclei)
    img_labeled_cytoplasm = iw.remove_small_regions(img_labeled_cytoplasm, min_size=min_size)
    img_cytoplasm_nuclei = mh.labeled.remove_bordering(img_labeled_cytoplasm)
    
    # Evaluate using F1 score
    f1 = f1_score(cytoplasm_labels.flatten(), img_cytoplasm_nuclei.flatten(), average='micro')
    
    return f1


In [8]:
study = optuna.create_study(
    storage="sqlite:///optuna.db",
    load_if_exists=True,
    study_name="cytoplasm_segmentation",
    direction="maximize"
    )
study.optimize(objective, n_trials=50)

[I 2024-11-07 17:50:06,128] Using an existing study with name 'cytoplasm_segmentation' instead of creating a new one.
[I 2024-11-07 17:50:12,773] Trial 1 finished with value: 0.7734947364560987 and parameters: {'slope': 19, 'limit': 1302, 'samples': 132, 'sigma': 7.205548764269908, 'subwidth': 17, 'subweight': 15, 'mask_size': 5, 'iterations': 7, 'min_size': 193}. Best is trial 1 with value: 0.7734947364560987.
[I 2024-11-07 17:50:37,236] Trial 2 finished with value: 0.7799605528882043 and parameters: {'slope': 18, 'limit': 1871, 'samples': 497, 'sigma': 14.06816245867328, 'subwidth': 14, 'subweight': 17, 'mask_size': 5, 'iterations': 2, 'min_size': 263}. Best is trial 2 with value: 0.7799605528882043.
[I 2024-11-07 17:50:46,031] Trial 3 finished with value: 0.7778362761507372 and parameters: {'slope': 17, 'limit': 1338, 'samples': 154, 'sigma': 8.040883550669287, 'subwidth': 14, 'subweight': 10, 'mask_size': 4, 'iterations': 3, 'min_size': 211}. Best is trial 2 with value: 0.779960552

In [9]:
print("Best parameters:", study.best_params)
print("Best F1 score:", study.best_value)

Best parameters: {'slope': 7, 'limit': 1623, 'samples': 423, 'sigma': 1.0056550229113084, 'subwidth': 21, 'subweight': 29, 'mask_size': 5, 'iterations': 7, 'min_size': 143}
Best F1 score: 0.7871034661595644
